In [4]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, top_k_accuracy_score, precision_score, recall_score, f1_score
import pickle
from node2vec import Node2Vec as n2v
import pandas as pd
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
warnings.filterwarnings("ignore")

# final code

In [5]:
#user set params
dir_path = "WSJ split data files"
node_feature_dir_path = "WSJ node features"
dbname = "WSJ"
epochs = 100

In [6]:
# converting nodes to continuous variable
train_df = pd.DataFrame()
for i in range(1,11,1):
    df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
    train_df = train_df.append(df, ignore_index=True)
    del df
temp_node_no = {}
id = 0
for i in np.unique(train_df[['src', 'dest']].values):
    temp_node_no[i] = id
    id += 1
print("total unique nodes found : ",id)
pickle.dump( temp_node_no, open( "node_id_into_continuous_node_ids.p", "wb" ) )
del train_df

total unique nodes found :  8536


In [7]:
for split_no in range(1,11,1):
    test_file_no = split_no
    print("\n\n\n Test file no : ",test_file_no)
    #creating training and testing dataset
    train_df = pd.DataFrame()
    #test_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        else:
            test_df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv")
            neg_df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
            test_df = test_df.append(neg_df, ignore_index=True)
            del neg_df
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    #test_df = test_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    test_df['src'] = [temp_node_no[i] for i in test_df['src']]
    test_df['dest'] = [temp_node_no[i] for i in test_df['dest']]
    
    print("Dataset has been successfully prepared")
    # creating training graph
    edge_list = [(train_df['src'][i],train_df['dest'][i]) for i in train_df.index]
    G = nx.MultiGraph()
    G.add_edges_from(edge_list)
    
    curr = time.time()
    #training n2v node embedding model
    g_emb = n2v(G, dimensions=64)

    WINDOW = 1 # Node2Vec fit window
    MIN_COUNT = 1 # Node2Vec min. count
    BATCH_WORDS = 4 # Node2Vec batch words

    mdl = g_emb.fit(
        window=WINDOW,
        min_count=MIN_COUNT,
        batch_words=BATCH_WORDS
    )
    
    timetaken = time.time()-curr
    print("time taken for training node embedding model : ",timetaken)
    mdl.save("my_"+str(dbname)+"_N2V_node_embedding_model_"+str(test_file_no)+".model")
    #adding negative edges to train_df
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
            train_df = train_df.append(df, ignore_index=True)
            del df
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    train_df.loc[train_df['label'] == -1, "label"] = 0
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    print(train_df.head)
    # creating x_train
    X_train = [(mdl.wv.get_vector(train_df['src'][i]) + mdl.wv.get_vector(train_df['dest'][i])) for i in train_df.index]
    X_train = np.array(X_train)
    y_train = np.array(train_df['label'])
    del train_df
    
    # creating x_test
    #test_df = test_df.sample(frac=1).reset_index(drop=True)
    #X_test = [(mdl.wv.get_vector(test_df['src'][i]) + mdl.wv.get_vector(test_df['dest'][i])) for i in test_df.index]
    #pickle.dump(X_test,open("n2v_"+str(dbname)+"_X_test_whole_"+str(test_file_no)+".csv","wb"))
    #X_test = np.array(X_test)
    #y_test = list(test_df['label'])
    del test_df
    
    print("x_train and x_test are ready")
    # creating keras model
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dense(1, activation='sigmoid'))

    curr = time.time()

    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, y_train, validation_split=0.10, epochs=epochs, batch_size=16,verbose=1)

    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    model.save("my_"+str(dbname)+"_N2V_model_"+str(test_file_no)+".keras")




 Test file no :  1
Dataset has been successfully prepared


Computing transition probabilities:   0%|          | 0/8536 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 10/10 [06:05<00:00, 36.52s/it]


time taken for training node embedding model :  3565.7120933532715
<bound method NDFrame.head of           src  dest  label
0        5176  6392      1
1         344  4836      0
2        5622  6594      1
3          88  4625      1
4        3577  7954      0
...       ...   ...    ...
1139395  3441  3189      0
1139396  7865  3993      0
1139397  2044  1424      0
1139398  6227   403      0
1139399  4952  6151      1

[1139400 rows x 3 columns]>
x_train and x_test are ready
Epoch 1/100


2023-07-19 16:56:54.977799: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-19 16:56:54.977840: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-19 16:56:54.977854: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a3d): /proc/driver/nvidia/version does not exist
2023-07-19 16:56:54.978079: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


...

Generating walks (CPU: 1): 100%|████████████████| 10/10 [06:08<00:00, 36.82s/it]


time taken for training node embedding model :  4054.587105512619
<bound method NDFrame.head of           src  dest  label
0          36  2575      1
1        2495  3836      0
2         591  2171      1
3        2381   413      0
4        5550  5564      1
...       ...   ...    ...
1139397  5773  5406      0
1139398  8105  8109      0
1139399  5683  6910      1
1139400  6084  7120      1
1139401  3456  4520      0

[1139402 rows x 3 columns]>
x_train and x_test are ready
Epoch 1/100
64092/64092 [==============================] - 224s 3ms/step - loss: 0.4102 - accuracy: 0.8254 - val_loss: 0.3926 - val_accuracy: 0.8352
Epoch 2/100
64092/64092 [==============================] - 220s 3ms/step - loss: 0.3866 - accuracy: 0.8372 - val_loss: 0.3850 - val_accuracy: 0.8388
Epoch 3/100
64092/64092 [==============================] - 222s 3ms/step - loss: 0.3811 - accuracy: 0.8394 - val_loss: 0.3836 - val_accuracy: 0.8404
Epoch 4/100
64092/64092 [==============================] - 222s 3ms/step - 

Computing transition probabilities:   0%|          | 0/8536 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 10/10 [04:24<00:00, 26.40s/it]


time taken for training node embedding model :  3597.5214138031006
<bound method NDFrame.head of           src  dest  label
0        5069  6945      1
1          91  2162      1
2        5715  5931      1
3         655  2548      0
4        5918  7522      0
...       ...   ...    ...
1139397   286  5977      1
1139398  2702  6214      0
1139399    60  1049      1
1139400  1697  1436      0
1139401  8284  7522      0

[1139402 rows x 3 columns]>
x_train and x_test are ready
Epoch 1/100
64092/64092 [==============================] - 430s 6ms/step - loss: 0.4116 - accuracy: 0.8240 - val_loss: 0.3922 - val_accuracy: 0.8338
Epoch 2/100
64092/64092 [==============================] - 351s 5ms/step - loss: 0.3879 - accuracy: 0.8359 - val_loss: 0.3833 - val_accuracy: 0.8374
Epoch 3/100
64092/64092 [==============================] - 344s 5ms/step - loss: 0.3814 - accuracy: 0.8385 - val_loss: 0.3811 - val_accuracy: 0.8371
Epoch 4/100
64092/64092 [==============================] - 347s 5ms/step -

# rough draft of code

In [4]:
dir_path = "WSJ split data files"
test_file_no = 1

In [5]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in range(1,11,1):
    if i != test_file_no:
        df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
        train_df = train_df.append(df, ignore_index=True)
        del df
    else:
        test_df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv")
        neg_df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
        test_df = test_df.append(neg_df, ignore_index=True)
        del neg_df
train_df

<ipython-input-5-ae5d777dec2f>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(neg_df, ignore_index=True)
<ipython-input-5-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-5-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-5-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-5-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

,src,dest,layer,weight,sign,label
0,5046,8395,12,2,-2.0,1
1,100036,103501,3,27,-2.0,1
2,100670,102593,3,2,-2.0,1
3,870,5764,2,33,-1.0,1
4,5036,6048,12,7,-2.0,1
...,...,...,...,...,...,...
569695,100442,100451,3,4,-2.0,1
569696,5338,8134,12,3,-2.0,1
569697,100075,100086,3,5,-2.0,1
569698,5082,5400,12,2,-2.0,1


In [6]:
train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)

In [11]:
test_df = test_df.drop(['layer', 'weight',"sign"], axis=1)
test_df.head()

,src,dest,label
0,100329,101979,1
1,100746,100788,1
2,5024,8029,1
3,100405,103390,1
4,100205,100259,1


In [8]:
# converting nodes to continuous variable
temp_node_no = {}
id = 0
for i in np.unique(train_df[['src', 'dest']].values):
    temp_node_no[i] = id
    id += 1
print("total unique nodes found till train_df: ",id)

total unique nodes found till train_df:  8536


In [12]:
train_df['src'] = [temp_node_no[i] for i in train_df['src']]
train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
test_df['src'] = [temp_node_no[i] for i in test_df['src']]
test_df['dest'] = [temp_node_no[i] for i in test_df['dest']]

In [15]:
# creating training graph
edge_list = [(train_df['src'][i],train_df['dest'][i]) for i in train_df.index]
G = nx.MultiGraph()
G.add_edges_from(edge_list)

In [21]:
del train_nx

In [20]:

g_emb = n2v(G, dimensions=16)

WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

print(emb_df.head())

Computing transition probabilities:   0%|          | 0/8536 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [06:35<00:00, 39.56s/it]


            0         1         2         3         4         5         6   \
74    0.066213 -0.087268 -0.160592  0.561564 -0.053954  0.517867  1.757313   
3423 -0.756973 -0.708204 -0.233035  0.840388  0.914142 -0.429827  1.208549   
4921 -0.090328 -0.784583  0.783312  0.461973  0.655650 -0.031383  0.340182   
8386 -0.102974 -0.580768 -0.139828  1.530549 -0.714675  0.597381  0.192185   
5555  0.756856 -1.469142  0.481359  0.932975  0.577899  0.809864  1.089535   

            7         8         9         10        11        12        13  \
74    0.057634  0.470260 -0.801055 -0.672506 -0.564250 -0.413633 -0.110114   
3423 -1.844603  0.637675  0.623564 -0.494655  0.312868  0.391652 -0.275188   
4921  0.009983 -0.715256 -0.478891 -0.807956  0.320143 -0.184014 -0.465778   
8386 -2.274364 -0.173700  0.053018 -2.846182  0.484979  1.219140 -1.585759   
5555 -0.271014 -1.360325  0.463972  0.147455 -0.262793  1.564479 -0.049630   

            14        15  
74   -0.373733 -1.602690  
3423 -0.

In [ ]:
3:27-3:31

In [34]:
import pickle
pickle.dump( mdl, open( "mdl_1.p", "wb" ) )

In [30]:
type(train_df['src'][1])

numpy.int64

In [22]:
del emb_df

# final code for creating training and testing data


In [24]:
mdl.wv.get_vector(74)

array([ 0.26640213, -0.61832184,  0.8166052 ,  0.4864245 ,  0.6819291 ,
        0.7843016 ,  0.61891556,  0.5080065 , -0.8787431 ,  0.0074712 ,
       -0.15300506, -0.2796761 , -0.12839389, -0.27083758, -0.18696621,
        0.35777766], dtype=float32)

In [46]:
train_df = pd.DataFrame()
test_file_no = 1
dir_path = "WSJ split data files"
for i in range(1,11,1):
    if i != test_file_no:
        df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
        train_df = train_df.append(df, ignore_index=True)
        df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
        train_df = train_df.append(df, ignore_index=True)
        del df
train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
train_df['src'] = [temp_node_no[i] for i in train_df['src']]
train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
train_df = train_df.sample(frac=1).reset_index(drop=True)

# creating x_train
X_train = [(mdl.wv.get_vector(train_df['src'][i]) + mdl.wv.get_vector(train_df['dest'][i])) for i in train_df.index]
X_train = np.array(X_train)
y_train = np.array(train_df['label'])

del train_df

<ipython-input-46-1a7f55301f01>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-46-1a7f55301f01>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-46-1a7f55301f01>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-46-1a7f55301f01>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-46-1a7f55301f01>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

In [33]:
test_df = test_df.sample(frac=1).reset_index(drop=True)

# creating x_train
X_test = [(mdl.wv.get_vector(test_df['src'][i]) + mdl.wv.get_vector(test_df['dest'][i])) for i in test_df.index]
X_test = np.array(X_test)
y_test = list(test_df['label'])

del test_df

In [37]:
len(X_train)*0.05

56970.0

In [38]:
X_val = X_train[0:56970]
y_val = y_train[0:56970]
X_val.shape

(56970, 16)

In [41]:
X_train = X_train[56970:]
y_train = y_train[56970:]

In [47]:
X_train.shape

(1139400, 16)

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
 
epochs = 100

model = Sequential()
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

curr = time.time()

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_split=0.10, epochs=epochs, batch_size=64,verbose=2)

timetaken = time.time()-curr
print("time taken for training : ",timetaken)
print("time taken for per epoch : ",timetaken/epochs)

Epoch 1/100
16023/16023 - 16s - loss: -3.0435e+05 - accuracy: 0.0274 - val_loss: -1.0635e+06 - val_accuracy: 0.0257 - 16s/epoch - 976us/step
Epoch 2/100
16023/16023 - 14s - loss: -3.2776e+06 - accuracy: 0.0261 - val_loss: -6.4844e+06 - val_accuracy: 0.0269 - 14s/epoch - 845us/step
Epoch 3/100
16023/16023 - 14s - loss: -1.2235e+07 - accuracy: 0.0261 - val_loss: -1.9426e+07 - val_accuracy: 0.0258 - 14s/epoch - 865us/step
Epoch 4/100
16023/16023 - 15s - loss: -3.0173e+07 - accuracy: 0.0257 - val_loss: -4.2837e+07 - val_accuracy: 0.0253 - 15s/epoch - 947us/step
Epoch 5/100
16023/16023 - 17s - loss: -6.0158e+07 - accuracy: 0.0255 - val_loss: -7.9737e+07 - val_accuracy: 0.0252 - 17s/epoch - 1ms/step
Epoch 6/100
16023/16023 - 18s - loss: -1.0520e+08 - accuracy: 0.0254 - val_loss: -1.3318e+08 - val_accuracy: 0.0246 - 18s/epoch - 1ms/step
Epoch 7/100
16023/16023 - 16s - loss: -1.6821e+08 - accuracy: 0.0251 - val_loss: -2.0602e+08 - val_accuracy: 0.0249 - 16s/epoch - 970us/step
Epoch 8/100
16023

Epoch 59/100
16023/16023 - 15s - loss: -1.0157e+11 - accuracy: 0.0242 - val_loss: -1.0393e+11 - val_accuracy: 0.0234 - 15s/epoch - 936us/step
Epoch 60/100
16023/16023 - 16s - loss: -1.0680e+11 - accuracy: 0.0242 - val_loss: -1.0924e+11 - val_accuracy: 0.0234 - 16s/epoch - 977us/step
Epoch 61/100
16023/16023 - 14s - loss: -1.1223e+11 - accuracy: 0.0242 - val_loss: -1.1475e+11 - val_accuracy: 0.0234 - 14s/epoch - 896us/step
Epoch 62/100
16023/16023 - 16s - loss: -1.1783e+11 - accuracy: 0.0242 - val_loss: -1.2042e+11 - val_accuracy: 0.0234 - 16s/epoch - 981us/step
Epoch 63/100
16023/16023 - 16s - loss: -1.2361e+11 - accuracy: 0.0242 - val_loss: -1.2628e+11 - val_accuracy: 0.0234 - 16s/epoch - 974us/step
Epoch 64/100
16023/16023 - 15s - loss: -1.2958e+11 - accuracy: 0.0242 - val_loss: -1.3233e+11 - val_accuracy: 0.0234 - 15s/epoch - 912us/step
Epoch 65/100
16023/16023 - 15s - loss: -1.3573e+11 - accuracy: 0.0242 - val_loss: -1.3857e+11 - val_accuracy: 0.0234 - 15s/epoch - 924us/step
Epoch 

In [50]:
model.save("n2v_classficatio_1")

INFO:tensorflow:Assets written to: n2v_classficatio_1/assets


In [51]:
model.save("my_N2V_model_1.keras")

In [ ]:
train_df = []
test_df = []
for file_no in range(1,11,1):
    